# Caching

> SAX Caching

In [2]:
import os, sys; sys.stderr = open(os.devnull, "w")

In [3]:
from __future__ import annotations

import gc
from functools import _lru_cache_wrapper, lru_cache, partial, wraps
from typing import Callable, Optional

In [4]:

_cached_functions = []

:::{eval-rst}
.. autofunction:: sax.caching.cache
:::


In [5]:

def cache(func: Optional[Callable] = None, *, maxsize: Optional[int] = None) -> Callable:
    """cache a function"""
    if func is None:
        return partial(cache, maxsize=maxsize)

    cached_func = lru_cache(maxsize=maxsize)(func)

    @wraps(func)
    def new_func(*args, **kwargs):
        return cached_func(*args, **kwargs)

    new_func.cache_clear = cached_func.cache_clear

    _cached_functions.append(new_func)

    return new_func

:::{eval-rst}
.. autofunction:: sax.caching.cache_clear
:::


In [6]:
def cache_clear(*, force: bool=False):
    """clear all function caches"""
    if not force:
        for func in _cached_functions:
            func.cache_clear()
    else:
        gc.collect()
        funcs = [a for a in gc.get_objects() if isinstance(a, _lru_cache_wrapper)]

        for func in funcs:
            func.cache_clear()